##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [1]:
!pip install -U -q google-generativeai

In [2]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

### Mount Google Drive

In [3]:
from google.colab import drive

drive.mount("/gdrive")

Mounted at /gdrive


## Set the API key

# New Section

Add your API_KEY to the secrets manager in the left panel "🔑".

In [4]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [5]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [6]:
model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = '...' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [7]:
gais_contents

[{'role': 'user',
  'parts': [{'file_data': {'mime_type': 'audio/mpeg',
     'drive_id': '1ADe-glXxiFeRiZ6cXMBG-vT1AtrOP0TP'}},
   {'text': 'Generate audio diarization, including transcriptions and speaker information for each transcription, for this interview. Organize the transcription by the time they happened.'}]},
 {'role': 'model',
  'parts': [{'text': "## Audio Diarization: \\n\\n**0:00-0:12**  Karen: Well, hello everyone. Helpfulness has always been at the heart of Google's mission and so since the outbreak of the COVID-19 pandemic, we wanted to share some of the ways that we've been helping. \\n\\n**0:12-0:35** Karen: We have launched more than 200 products and features and contributed more than a $billion in resources to help our users, our partners, everyone to get through this pandemic. I want to highlight a few of the areas we've been focused on before we broaden the conversation. You've probably heard the news about Google's collaboration with Apple. We announced this las

In [8]:
generation_config

{'temperature': 1, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 8192}

In [9]:
safety_settings

{}

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [10]:
# @title `show_file` function
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [12]:
def show_file(file_data):
    mime_type = file_data["mime_type"]
    data = None  # Initialize data to None

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes() #Consider reading the data here if needed
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content #Consider reading the data here if needed
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        # Check if data is available before accessing its length
        if data is not None and len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        # Check if data is available before accessing its length
        if data is not None and len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [ ]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [15]:
# Import necessary modules
import copy
import pathlib
import hashlib
import requests
!pip install google-generativeai # Install the genai library
import google.generativeai as genai # Import the library using its correct name

# Define upload_file_data (assuming it's in ipython-input-0-73f0db0e92c1)

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type) # Assuming 'path' is defined before
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

# Now you can use upload_file_data in other cells:

contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Uploading: /gdrive/.shortcut-targets-by-id/1ADe-glXxiFeRiZ6cXMBG-vT1AtrOP0TP/generative-ai_audio_audio_covid_google_response.mp3


Here is the coverted `Content`s:

In [ ]:
contents

## Call `generate_content`

In [16]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [17]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

Let me know if you want to know more about any of these steps! 


In [18]:
response.candidates

[content {
  parts {
    text: "Let me know if you want to know more about any of these steps! \n"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

## Or Create a chat

In [19]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [20]:
model_turn = contents[-1].get("role", None) == "user"

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])

    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

In [21]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [22]:
response = chat.send_message("data science grow my carrer or not.")
display(Markdown(response.text))

Data science is a fantastic field for career growth, but it's not a guaranteed success.  Here's a breakdown to help you decide:

**Why Data Science is a Great Career Choice:**

* **High Demand:**  Data scientists are in high demand across industries.  Every company is generating vast amounts of data, and they need experts to make sense of it.
* **Good Pay:** Data science careers tend to be well-compensated, with starting salaries often above average and plenty of room for growth.
* **Interesting Work:** You'll get to work on projects that have a real impact, solving problems and answering questions that drive business decisions.
* **Constant Learning:** Data science is an evolving field, meaning you'll always have opportunities to learn new skills and technologies.

**But, There are Challenges:**

* **Competitive:**  The field is competitive, so you need to stand out with strong skills and experience.
* **Ever-Changing Technology:** You'll need to stay up-to-date with the latest tools and techniques. 
* **Problem-Solving Focus:**  Data science requires a strong analytical mind and a passion for solving problems.
* **Not Just a "Coding" Job:**  While coding skills are important, data science also involves communication, collaboration, and understanding business needs.

**To Grow Your Career in Data Science:**

1. **Develop Strong Skills:**  Focus on essential skills like programming (Python, R), statistics, machine learning, data visualization, and SQL. 
2. **Get Real-World Experience:**  Build a portfolio through projects, internships, or volunteering, showcasing your abilities.
3. **Networking:** Connect with other data professionals to learn, share experiences, and explore opportunities.
4. **Stay Updated:**  Continuously learn new technologies and keep up with industry trends.
5. **Communication:**  Develop strong communication skills to explain complex technical concepts to non-technical audiences.

**Ultimately, whether data science will grow your career depends on you.** Are you passionate about the field, willing to learn, and driven to excel? If so, data science offers a very promising path for a successful and fulfilling career. 
